In [1]:
import pandas as pd
from datetime import timedelta
from bisect import bisect_left


df = pd.read_csv('data/305_user_logs/log.csv')

# 1 question
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d_%H:%M:%S")
df_sorted = df.sort_values(['user', 'date'])
df_sorted['prev_time'] = df_sorted.groupby('user')['date'].shift()
df_sorted['new_session'] = (df_sorted['date'] - df_sorted['prev_time'] >= timedelta(minutes=30)) | df_sorted['prev_time'].isna()
df_sorted['session_id'] = df_sorted.groupby('user')['new_session'].cumsum()
sessions_start = df_sorted.groupby(['user', 'session_id'])['date'].min().reset_index()
sessions_count = (sessions_start['date'].dt.date == pd.to_datetime("2020-04-19").date()).sum()

# 2 question
video_users = df[(df['event_type'] == 2) & (df['parameter'] == 'video')]
video_users['day'] = video_users['date'].dt.date
video_unique = video_users.groupby('day')['user'].nunique()
max_video_day = video_unique.max()

# 3 question
df_sorted_time = df.sort_values('date')
times = df_sorted_time['date'].tolist()
n = len(times)
max_count, best_start = 0, None

for i in range(n):
    start_time = times[i]
    end_time = start_time + timedelta(minutes=5)
    j = bisect_left(times, end_time)
    count = j - i
    if count >= max_count:
        max_count = count
        best_start = start_time
best_start_str = best_start.strftime("%Y-%m-%d_%H:%M:%S")

print(sessions_count, max_video_day, best_start_str)

C:\Users\xz1v\AppData\Local\Temp\ipykernel_316\1695538941.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_users['day'] = video_users['date'].dt.date


6251 391 2020-04-09_13:06:42
